In [1]:
import os
import numpy as np
import tensorflow as tf

from PIL import Image
from stylegan2.utils import postprocess_images
from load_models import load_generator
from copy_official_weights import convert_official_weights_together
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))


Num GPUs Available:  1


In [2]:
from tf_utils import allow_memory_growth
allow_memory_growth()
ckpt_dir_base = './official-converted'
# saving phase
for use_custom_cuda in [True, False]:
    ckpt_dir = os.path.join(ckpt_dir_base, 'cuda') if use_custom_cuda else os.path.join(ckpt_dir_base, 'ref')
    convert_official_weights_together(ckpt_dir, use_custom_cuda)

1 Physical GPUs, 1 Logical GPUs
Setting up TensorFlow plugin "upfirdn_2d.cu": Preprocessing... Failed!


RuntimeError: NVCC returned an error. See below for full command line and output log:

nvcc --std=c++11 -DNDEBUG "/media/charles/SSD/sourcecode/f2f/stylegan2-tf-2.x/stylegan2/layers/cuda/upfirdn_2d.cu" --preprocess -o "/tmp/tmpmdju7itv/upfirdn_2d_tmp.cu" --keep --keep-dir "/tmp/tmpmdju7itv" --disable-warnings --include-path "/home/charles/miniconda3/envs/deepIoM-tf2/lib/python3.7/site-packages/tensorflow_core/include" --include-path "/home/charles/miniconda3/envs/deepIoM-tf2/lib/python3.7/site-packages/tensorflow_core/include/external/protobuf_archive/src" --include-path "/home/charles/miniconda3/envs/deepIoM-tf2/lib/python3.7/site-packages/tensorflow_core/include/external/com_google_absl" --include-path "/home/charles/miniconda3/envs/deepIoM-tf2/lib/python3.7/site-packages/tensorflow_core/include/external/eigen_archive" 2>&1

In file included from /usr/local/cuda-10.1/bin/../targets/x86_64-linux/include/cuda_runtime.h:83,
                 from <command-line>:
/usr/local/cuda-10.1/bin/

In [3]:
def test_generator(ckpt_dir, use_custom_cuda):
    g_clone = load_generator(g_params=None, is_g_clone=True, ckpt_dir=ckpt_dir, custom_cuda=use_custom_cuda)

    # test
    seed = 6600
    rnd = np.random.RandomState(seed)
    latents = rnd.randn(1, g_clone.z_dim)
    labels = rnd.randn(1, g_clone.labels_dim)
    latents = latents.astype(np.float32)
    labels = labels.astype(np.float32)
    image_out = g_clone([latents, labels], training=False, truncation_psi=0.5)
    image_out = postprocess_images(image_out)
    image_out = image_out.numpy()

#     out_fn = f'seed{seed}-{out_fn}'
#     Image.fromarray(image_out[0], 'RGB').save(out_fn)
    return image_out


In [4]:

    
# inference phase
ckpt_dir_cuda = os.path.join(ckpt_dir_base, 'cuda')
ckpt_dir_ref = os.path.join(ckpt_dir_base, 'ref')

# 1. inference cuda saved weight from cuda model
test_generator(ckpt_dir_cuda, use_custom_cuda=True, out_fn='from-cuda-to-cuda.png')

# 2. inference cuda saved weight from ref model
test_generator(ckpt_dir_cuda, use_custom_cuda=False, out_fn='from-cuda-to-ref.png')

# 3. inference ref saved weight from ref model
test_generator(ckpt_dir_ref, use_custom_cuda=False, out_fn='from-ref-to-ref.png')

# 4. inference ref saved weight from cuda model
test_generator(ckpt_dir_ref, use_custom_cuda=True, out_fn='from-ref-to-cuda.png')


TypeError: test_generator() got an unexpected keyword argument 'out_fn'